# Determinantal Point Process in Python

First, let's start the environment.

In [14]:
import numpy as np
import matplotlib.pyplot as plt

# This is a bit of magic to make matplotlib figures appear inline in the notebook
# rather than in a new window.
%matplotlib inline
plt.rcParams['figure.figsize'] = (10.0, 8.0) # set default size of plots
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

# Some more magic so that the notebook will reload external python modules;
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Questions

1. How do I define a $L$ kernel? Can it be defined through $B^T B$?
2. The kernel $K$ can be seen as a cumulative function of $L$, since it computes $P_K(A \subseteq \mathbf{Y})$ instead of $P_L(\mathbf{Y} = A)$?

In [15]:
# Understanding broadcasting, used by Kulesza in the MATLAB code (bsxfun)
A = np.arange(0,10) 
print A[np.newaxis].T
print A
print A[np.newaxis].T - A
# You're taking the first elem, say [0], you fill it until it reaches the same rank of A, then you subtract [0] by
# every element in A

# it's like I stacked A[np.newaxis].T until it had 9 dims
print ""
print np.tile(A[np.newaxis].T, (1,10))
print np.tile(A, (10,1)) 
print np.tile(A[np.newaxis].T, (1,10)) - np.tile(A, (10,1))

[[0]
 [1]
 [2]
 [3]
 [4]
 [5]
 [6]
 [7]
 [8]
 [9]]
[0 1 2 3 4 5 6 7 8 9]
[[ 0 -1 -2 -3 -4 -5 -6 -7 -8 -9]
 [ 1  0 -1 -2 -3 -4 -5 -6 -7 -8]
 [ 2  1  0 -1 -2 -3 -4 -5 -6 -7]
 [ 3  2  1  0 -1 -2 -3 -4 -5 -6]
 [ 4  3  2  1  0 -1 -2 -3 -4 -5]
 [ 5  4  3  2  1  0 -1 -2 -3 -4]
 [ 6  5  4  3  2  1  0 -1 -2 -3]
 [ 7  6  5  4  3  2  1  0 -1 -2]
 [ 8  7  6  5  4  3  2  1  0 -1]
 [ 9  8  7  6  5  4  3  2  1  0]]

[[0 0 0 0 0 0 0 0 0 0]
 [1 1 1 1 1 1 1 1 1 1]
 [2 2 2 2 2 2 2 2 2 2]
 [3 3 3 3 3 3 3 3 3 3]
 [4 4 4 4 4 4 4 4 4 4]
 [5 5 5 5 5 5 5 5 5 5]
 [6 6 6 6 6 6 6 6 6 6]
 [7 7 7 7 7 7 7 7 7 7]
 [8 8 8 8 8 8 8 8 8 8]
 [9 9 9 9 9 9 9 9 9 9]]
[[0 1 2 3 4 5 6 7 8 9]
 [0 1 2 3 4 5 6 7 8 9]
 [0 1 2 3 4 5 6 7 8 9]
 [0 1 2 3 4 5 6 7 8 9]
 [0 1 2 3 4 5 6 7 8 9]
 [0 1 2 3 4 5 6 7 8 9]
 [0 1 2 3 4 5 6 7 8 9]
 [0 1 2 3 4 5 6 7 8 9]
 [0 1 2 3 4 5 6 7 8 9]
 [0 1 2 3 4 5 6 7 8 9]]
[[ 0 -1 -2 -3 -4 -5 -6 -7 -8 -9]
 [ 1  0 -1 -2 -3 -4 -5 -6 -7 -8]
 [ 2  1  0 -1 -2 -3 -4 -5 -6 -7]
 [ 3  2  1  0 -1 -2 -3 -4 -5 -6]
 

In [16]:
n = 10 # thus, N = n^2. As I'm sampling from a plane, D = 2
sigma = 0.1 
grid_points = np.arange(n) / float(n)
xx, yy = np.meshgrid(grid_points, grid_points)

print "\nxx.T - xx\n", (xx.T - xx)
print "\n(xx.T - xx)**2\n",(xx.T - xx)**2 
print "\n(xx.T - xx)**2 + (yy.T - yy)**2\n", (xx.T - xx)**2 + (yy.T - yy)**2 
print "\nnp.exp((xx.T - xx)**2 + (yy.T - yy)**2)\n", np.exp((xx.T - xx)**2 + (yy.T - yy)**2)
print "\nnp.exp(-0.5 * ((xx.T - xx)**2 + (yy.T - yy)**2))\n", np.exp(-0.5 * ((xx.T - xx)**2 + (yy.T - yy)**2))
print "\nnp.exp(-0.5 * ((xx.T - xx)**2 + (yy.T - yy)**2) / sigma**2)\n", np.exp(-0.5 * ((xx.T - xx)**2 + (yy.T - yy)**2) / sigma**2)
L = np.exp(-0.5 * ((xx.T - xx)**2 + (yy.T - yy)**2) / sigma**2)


xx.T - xx
[[ 0.  -0.1 -0.2 -0.3 -0.4 -0.5 -0.6 -0.7 -0.8 -0.9]
 [ 0.1  0.  -0.1 -0.2 -0.3 -0.4 -0.5 -0.6 -0.7 -0.8]
 [ 0.2  0.1  0.  -0.1 -0.2 -0.3 -0.4 -0.5 -0.6 -0.7]
 [ 0.3  0.2  0.1  0.  -0.1 -0.2 -0.3 -0.4 -0.5 -0.6]
 [ 0.4  0.3  0.2  0.1  0.  -0.1 -0.2 -0.3 -0.4 -0.5]
 [ 0.5  0.4  0.3  0.2  0.1  0.  -0.1 -0.2 -0.3 -0.4]
 [ 0.6  0.5  0.4  0.3  0.2  0.1  0.  -0.1 -0.2 -0.3]
 [ 0.7  0.6  0.5  0.4  0.3  0.2  0.1  0.  -0.1 -0.2]
 [ 0.8  0.7  0.6  0.5  0.4  0.3  0.2  0.1  0.  -0.1]
 [ 0.9  0.8  0.7  0.6  0.5  0.4  0.3  0.2  0.1  0. ]]

(xx.T - xx)**2
[[ 0.    0.01  0.04  0.09  0.16  0.25  0.36  0.49  0.64  0.81]
 [ 0.01  0.    0.01  0.04  0.09  0.16  0.25  0.36  0.49  0.64]
 [ 0.04  0.01  0.    0.01  0.04  0.09  0.16  0.25  0.36  0.49]
 [ 0.09  0.04  0.01  0.    0.01  0.04  0.09  0.16  0.25  0.36]
 [ 0.16  0.09  0.04  0.01  0.    0.01  0.04  0.09  0.16  0.25]
 [ 0.25  0.16  0.09  0.04  0.01  0.    0.01  0.04  0.09  0.16]
 [ 0.36  0.25  0.16  0.09  0.04  0.01  0.    0.01  0.04  0.09]
 

## Computing probabilities using the ensemble kernel $L$

First, computing individual probabilities.

In [17]:
# Computing some probabilities using L-ensembles

# computing the probability of a singleton {i}, i.e., a set containing a single element
# np.linalg.det(L[0,0])
print "Pr(Y = 0) =", L[0,0]
print "Pr(Y = 1) =", L[1,1]

Pr(Y = 0) = 1.0
Pr(Y = 1) = 1.0


Probabilities of more elements.

In [18]:
print L[0:2, 0:2]
print np.linalg.det(L[0:2, 0:2])
print "by hand:", 1*1 - 0.36787944**2

print "\n", L[0:3, 0:3]
print np.linalg.det(L[0:3, 0:3])

print "\n", L
print np.linalg.det(L)

#print "\n", L[3, 3]
#print np.linalg.det(L[0:3, 0:3])


[[ 1.          0.36787944]
 [ 0.36787944  1.        ]]
0.864664716763
by hand: 0.864664717625

[[ 1.          0.36787944  0.01831564]
 [ 0.36787944  1.          0.36787944]
 [ 0.01831564  0.36787944  1.        ]]
0.733951475252

[[  1.00000000e+00   3.67879441e-01   1.83156389e-02   1.23409804e-04
    1.12535175e-07   1.38879439e-11   2.31952283e-16   5.24288566e-22
    1.60381089e-28   6.63967720e-36]
 [  3.67879441e-01   1.00000000e+00   3.67879441e-01   1.83156389e-02
    1.23409804e-04   1.12535175e-07   1.38879439e-11   2.31952283e-16
    5.24288566e-22   1.60381089e-28]
 [  1.83156389e-02   3.67879441e-01   1.00000000e+00   3.67879441e-01
    1.83156389e-02   1.23409804e-04   1.12535175e-07   1.38879439e-11
    2.31952283e-16   5.24288566e-22]
 [  1.23409804e-04   1.83156389e-02   3.67879441e-01   1.00000000e+00
    3.67879441e-01   1.83156389e-02   1.23409804e-04   1.12535175e-07
    1.38879439e-11   2.31952283e-16]
 [  1.12535175e-07   1.23409804e-04   1.83156389e-02   3.678794

In [19]:
# taking all but one column trick
A = np.array([[1,2,3,1],[4,5,6,2],[7,8,9,3],[0,0,0,4]])
print A
A[ [[0],[2],[3]] , [0,2,3] ]

[[1 2 3 1]
 [4 5 6 2]
 [7 8 9 3]
 [0 0 0 4]]


array([[1, 3, 1],
       [7, 9, 3],
       [0, 0, 4]])

In [99]:
print "i = {0,1,2}:", 
#print L[ [[0],[1],[2]] , [0,1,2] ]
print np.linalg.det(L[ [[0],[1],[2]] , [0,1,2] ])

print "i = {0,1,3}:", 
#print L[ [[0],[1],[3]] , [0,1,3] ]
print np.linalg.det(L[ [[0],[1],[3]] , [0,1,3] ])

print "i = {0,1,4}:", 
#print L[ [[0],[1],[4]] , [0,1,4] ]
print np.linalg.det(L[ [[0],[1],[4]] , [0,1,4] ])

print "i = {0,1,9}:", 
#print L[ [[0],[1],[9]] , [0,1,9] ]
print np.linalg.det(L[ [[0],[1],[9]] , [0,1,9] ])

print "\nAs expected, the set {0,1,2} has lower probability than other sets with more distant numbers,",
print "such as {0,1,3}, {0,1,9}"

i = {0,1,2}: 0.733951475252
i = {0,1,3}: 0.864330901963
i = {0,1,4}: 0.864664701544
i = {0,1,9}: 0.864664716763

As expected, the set {0,1,2} has lower probability than other sets with more distant numbers, such as {0,1,3}, {0,1,9}


In [21]:
# coding a way to use only the elements that i'm interested
A = np.array([[1,2,3,1],[4,5,6,2],[7,8,9,3],[0,0,0,4]])
selected_items = [0,3]
#br_idxs = map(lambda x: list(x), selected_items) # broadcasted items
br_idxs = [[a] for a in selected_items]

print br_idxs
print A[br_idxs, selected_items]

[[0], [3]]
[[1 1]
 [0 4]]


## Computing kernel $K$ from ensemble $L$

There are two ways to compute the marginal kernel $K$. Using the following operation:

$K = L (L+I)^{-1}$.

Or using the eigendecomposition of $L$:

$K = \sum_{n=1}^N{( \frac{\lambda_n}{\lambda_n + 1} \mathbf{v_n} \mathbf{v_n}^T )}$.

In [93]:
import scipy.linalg

# Using the first form:
K = L.dot( L + np.eye(L.shape[0], L.shape[1]) )
print "K =", K


# or using eigendecomposition:
#eV, eD = scipy.linalg.eig(L)
#
# (eV / (eV + 1.0))
#Lrecomp = np.zeros_like(eD)
#for i in xrange(eD.shape[0]):
#    print (eV[i]/(eV[i] + 1))
#    print (eD[i].dot(eD[i].T))
#    Lrecomp[i,:] = (eV[i]/(eV[i] + 1)) * eD[i].dot(eD[i].T)



K = [[  2.13567076e+00   1.11037853e+00   1.90327602e-01   1.38461648e-02
    4.26600098e-04   4.60349923e-06   1.93625059e-08   2.82847915e-11
    1.61004795e-14   3.18303396e-18]
 [  1.11037853e+00   2.27100604e+00   1.11711648e+00   1.90373002e-01
    1.38462062e-02   4.26600103e-04   4.60349923e-06   1.93625059e-08
    2.82847915e-11   1.61004795e-14]
 [  1.90327602e-01   1.11711648e+00   2.27134151e+00   1.11711874e+00
    1.90373004e-01   1.38462062e-02   4.26600103e-04   4.60349923e-06
    1.93625059e-08   2.82847915e-11]
 [  1.38461648e-02   1.90373002e-01   1.11711874e+00   2.27134152e+00
    1.11711874e+00   1.90373004e-01   1.38462062e-02   4.26600103e-04
    4.60349923e-06   1.93625059e-08]
 [  4.26600098e-04   1.38462062e-02   1.90373004e-01   1.11711874e+00
    2.27134152e+00   1.11711874e+00   1.90373004e-01   1.38462062e-02
    4.26600103e-04   4.60349923e-06]
 [  4.60349923e-06   4.26600103e-04   1.38462062e-02   1.90373004e-01
    1.11711874e+00   2.27134152e+00   1.1

In [97]:
# computing the same previous probabilities, but using the marginal kernel K
def rev_idxs(items):
    idxs_list = [[a] for a in items]
    return idxs_list, items

print "i = {0,1,2}:",
print np.linalg.det(K[ rev_idxs([0,1,2]) ])

print "i = {0,1,3}:",
print np.linalg.det(K[ rev_idxs([0,1,3]) ])

print "i = {0,1,4}:",
print np.linalg.det(K[ rev_idxs([0,1,4]) ])

print "i = {0,1,9}:",
print np.linalg.det(K[ rev_idxs([0,1,9]) ])

i = {0,1,2}: 5.94055091936
i = {0,1,3}: 8.14387043627
i = {0,1,4}: 8.21545603373
i = {0,1,9}: 7.72510711274



As ensemble $L$, notice that $K$ is not necessarily normalized, as it is also a state of proportionality. Also, $K$ computes a different probability of $L$. $K$ comprises the marginal probabilities of inclusion for subsets $A$, while $L$ specifies atomic probabilities for every possible instantiation of $\mathbf{Y}$.

## Dummy sampling algorithm

A dummy sampling approach consists of sampling an element a subset/element $A = {i}$ with probability $P(A = \{i\})$. After sampling the first element, we recompute the probabilities, but conditioning on the value that was previously sampled. Kulesza et al. presents a better approach based on the eigendecomposition, that we shall see further ahead.

In this sampling problem, we shall consider the same example of sampling a set of points $\{1,2,\dots,n\}$ from a 2D grid. Reintroducing the data:

In [108]:
n = 10 # thus, N = n^2. As I'm sampling from a plane, D = 2
sigma = 0.1 
grid_points = np.arange(n) / float(n)

[ 0.   0.1  0.2  0.3  0.4  0.5  0.6  0.7  0.8  0.9]


Defining functions to compute probabilities:

In [116]:
class DPP:
    def __init__(self, grid_points):
        self.xx, self.yy = np.meshgrid(grid_points, grid_points)
        self.L = np.exp(-0.5 * ((xx.T - xx)**2 + (yy.T - yy)**2) / sigma**2)

    def _idxs(self, items):
        idxs_list = [[a] for a in items]
        return idxs_list, items

    def _rev_idxs(self, items):
        # selects the opposite of the items specified
        pass
    
    def prob(self, items):
        return np.linalg.det( L[ self._idxs(items) ] )
    
    def cond_prob(self, items, sampled):
        """TODO implement the conditioning function"""
        np.linalg.det( L[ self._idxs(items) ] )
        pass
    
    def sample(self):
        pass
        

dpp_grid = DPP(grid_points)
print "sanity check"
print "i = {1,2,3}:", dpp_grid.prob([1,2,3])

sanity check
i = {1,2,3}: 0.733951475252
